In [1]:
import tensorflow as tf

2023-12-26 19:24:29.136631: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-26 19:24:29.136676: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-26 19:24:29.138203: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-26 19:24:29.264973: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<h3 style='color:purple'>Create tf dataset from a list</h3>

In [2]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

2023-12-26 19:24:32.575810: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-26 19:24:32.602748: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-26 19:24:32.602937: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

<h3 style='color:purple'>Iterate through tf dataset</h3>

In [3]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


<h3 style='color:purple'>Iterate through elements as numpy elements</h3>

In [4]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


<h3 style='color:purple'>Iterate through first n elements in tf dataset</h3>

In [5]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


<h3 style='color:purple'>Filter sales numbers that are < 0</h3>

In [6]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


<h3 style='color:purple'>Convert sales numbers from USA dollars ($) to Indian Rupees (INR) Assuming 1->72 conversation rate</h3>

In [7]:
tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


<h3 style='color:purple'>Shuffe</h3>

In [8]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1584
2232
2304
1512
2448
2232


https://stackoverflow.com/questions/53514495/what-does-batch-repeat-and-shuffle-do-with-tensorflow-dataset

<h3 style='color:purple'>Batching</h3>

In [9]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1584 1512]
[2232 2448]
[2304 2232]


<h3 style='color:purple'>Perform all of the above operations in one shot</h3>

In [10]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1512 2232]
[1584 2304]
[2448 2232]


<h3>Images</h3>

In [33]:
images_ds = tf.data.Dataset.list_files('images/*/*', shuffle=False)

print('images_ds:')
for file in images_ds.take(10):
    print(file.numpy())

images_ds:
b'images/cat/20 Reasons Why Cats Make the Best Pets....jpg'
b'images/cat/7 Foods Your Cat Can_t Eat.jpg'
b'images/cat/A cat appears to have caught the....jpg'
b'images/cat/Adopt-A-Cat Month\xc2\xae - American Humane....jpg'
b'images/cat/All About Your Cat_s Tongue.jpg'
b'images/cat/Alley Cat Allies _ An Advocacy....jpg'
b'images/cat/Are Cats Domesticated_ _ The New Yorker.jpg'
b'images/cat/Cat Advice _ Collecting a Urine Sample....jpg'
b'images/cat/Cat Throwing Up_ Normal or Cause for....jpg'
b'images/cat/Cat intelligence - Wikipedia.jpg'


In [15]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'images/dog/The History of Dogs as Pets - ABC News.jpg'
b'images/dog/20 must-have products for new dog owners.jpg'
b'images/dog/50 dog breeds and their history that....jpg'


In [12]:
image_count = len(images_ds)
image_count

130

In [13]:
type(images_ds)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [14]:
for file in images_ds.take(3):
    print(file.numpy())

b'images/cat/20 Reasons Why Cats Make the Best Pets....jpg'
b'images/cat/7 Foods Your Cat Can_t Eat.jpg'
b'images/cat/A cat appears to have caught the....jpg'


In [16]:
class_names = ["cat","dog"]

In [17]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [18]:
len(train_ds)

104

In [19]:
len(test_ds)

26

In [36]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    #print('parts:', parts)
    # parts: tf.Tensor([b'images' b'dog' b'20 Reasons Why Cats Make the Best Pets....jpg'], shape=(3,), dtype=string)
    #print('parts[-1]:', parts[-1]) 
    # parts[-1]: tf.Tensor(b'20 Reasons Why Cats Make the Best Pets....jpg', shape=(), dtype=string)
    #print('parts[-2]:', parts[-2])
    # parts[-2]: tf.Tensor(b'dog', shape=(), dtype=string)
    #print('parts[-3]:', parts[-3])
    # parts[-3]: tf.Tensor(b'images', shape=(), dtype=string)
    return parts[-2] # 'dog'

In [21]:
get_label("images/dog/20 Reasons Why Cats Make the Best Pets....jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'dog'>

In [37]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [38]:
img, label = process_image("images/cat/20 Reasons Why Cats Make the Best Pets....jpg")
img.numpy()[:2]

array([[[155.      , 186.      , 215.      ],
        [156.      , 187.      , 216.      ],
        [158.      , 189.      , 218.      ],
        [160.0039  , 189.0039  , 219.0039  ],
        [161.0039  , 190.0039  , 220.0039  ],
        [162.      , 191.      , 221.      ],
        [166.      , 193.      , 222.      ],
        [167.      , 194.      , 223.      ],
        [168.      , 195.      , 224.      ],
        [169.      , 196.      , 225.      ],
        [170.      , 197.      , 224.      ],
        [170.0039  , 197.0039  , 224.0039  ],
        [172.      , 199.      , 226.      ],
        [173.0039  , 199.0039  , 224.0039  ],
        [174.0039  , 200.0039  , 225.0039  ],
        [175.0039  , 201.0039  , 226.0039  ],
        [176.      , 202.      , 227.      ],
        [177.0039  , 203.0039  , 228.0039  ],
        [177.0039  , 203.0039  , 228.0039  ],
        [179.0039  , 203.0039  , 227.0039  ],
        [180.0039  , 204.0039  , 228.0039  ],
        [180.0039  , 204.0039  , 2

In [24]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [25]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[ 36.039062   22.039062    9.0390625]
  [ 67.64844    40.523438   16.960938 ]
  [ 58.25       36.125      12.125    ]
  ...
  [247.        247.        247.       ]
  [247.        247.        247.       ]
  [247.        247.        247.       ]]

 [[ 30.282715   18.282715    6.282715 ]
  [ 66.30127    39.17627    16.606934 ]
  [ 63.070312   40.945312   17.882812 ]
  ...
  [247.        247.        247.       ]
  [247.        247.        247.       ]
  [247.        247.        247.       ]]

 [[ 29.887695   19.195312    8.50293  ]
  [ 61.234863   35.672363   13.109863 ]
  [ 59.199707   37.012207   14.074707 ]
  ...
  [247.        247.        247.       ]
  [247.        247.        247.       ]
  [247.        247.        247.       ]]

 ...

 [[ 31.992188   23.992188   12.9921875]
  [ 33.694824   25.694824   14.694824 ]
  [ 31.195312   22.195312   13.1953125]
  ...
  [ 65.         37.         23.       ]
  [ 64.10986    38.109863   25.109863 ]
  [ 66.         38.         

In [26]:
def scale(image, label):
    return image/255, label

In [27]:
train_ds = train_ds.map(scale)

In [28]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.2564951  0.31924018 0.421201  ]
****Label:  b'dog'
****Image:  [0.14386488 0.06984528 0.06285999]
****Label:  b'cat'
****Image:  [0.38039216 0.16470589 0.52156866]
****Label:  b'cat'
****Image:  [0.6240809  0.61623776 0.56917894]
****Label:  b'dog'
****Image:  [0.21355699 0.3477788  0.1400582 ]
****Label:  b'dog'
